In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import gstools as gs
from sklearn.metrics import mean_squared_error

In [20]:
# from functions import load_data
# This file stores the various custom-defined functions 
def load_data(file_name, protein):        
    # Specify the file path
    if file_name == "tonsil":
        file_path = "/Users/cui/Library/CloudStorage/OneDrive-YaleUniversity/0 High-Dim Spatial/Human Tonsil/tonsil_codex.csv"
    if file_name == "hubmap":
        file_path = "/Users/cui/Library/CloudStorage/OneDrive-YaleUniversity/0 High-Dim Spatial/hubmap/ann/B009A_22_03_03_Skywalker_reg001_compensated_ann.csv"
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df = df.head(20000)
    if file_name == "tonsil":
        df.rename(columns={'centroid_x': 'x'}, inplace=True)
        df.rename(columns={'centroid_y': 'y'}, inplace=True)
    
    # shuffle the rows
    df = df.sample(frac=1).reset_index(drop=True)
    
    # we normalise x and y so that the grid is approximately 1 by 1
    df['x'] = df['x']/df['x'].max()
    df['y'] = df['y']/df['y'].max()

    # we normalise the protein value and do a standard log transform for potential long-tail effects.
    df[protein] = df[protein] / df[protein].median()
    df[protein] = np.log(1+df[protein])
    #Pick only the large-value outliers, but we don't want to work with those that only give the value = 1.0

    top_20000_rows = df.nlargest(20000, protein)
    top_20000_rows = top_20000_rows.sample(frac=1).reset_index(drop=True)
    df_train = top_20000_rows.head(2000)
    df_test = top_20000_rows.tail(2000)
    
    x_train = df_train['x']
    y_train = df_train['y']
    val_train = df_train[protein]
    
    x_test = df_test['x']
    y_test = df_test['y']
    val_test = df_test[protein]

    return (x_train, y_train, val_train, x_test, y_test, val_test)

In [21]:
protein = "CD3"
x_train = 0 
x_train, y_train, val_train, x_test, y_test, val_test = load_data("tonsil", protein)

In [22]:
x_train

0       0.856185
1       0.555260
2       0.967147
3       0.628710
4       0.895694
          ...   
1995    0.599877
1996    0.656922
1997    0.815844
1998    0.500018
1999    0.396025
Name: x, Length: 2000, dtype: float64

In [18]:
type(x_train)

pandas.core.series.Series